# RNN Text Generation

This notebook implements a recurrent neural network that learns to compose sonnets after being trained by Shakespeare. A character level approach is used. Hidden layers use LSTM units. 

In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests

Using TensorFlow backend.


## Shakespeare's Sonnets or Frankenstein

A previously processed text file containing all of Shakespeare's sonnets or the text of Frankenstein by Mary Shelley is imported. The character vocabulary and dictionaries to make characters to indexes and vice versa are created.

In [2]:
# read in preprocessed text of shakespeare's sonnets 

# from local file system
# filename = 'sonnets.txt'
# filename = 'frankenstein.txt'
# file = open(filename,'r')
# text = file.read()

# from github
url = 'https://raw.githubusercontent.com/prof-groff/deep-learning/master/data/sonnets.txt'
# url = 'https://raw.githubusercontent.com/prof-groff/deep-learning/master/data/frankenstein.txt'
text = requests.get(url).text

len_text = len(text)
print('text length: '.upper() + str(len(text)) + '\n')
print('text sample:'.upper() + '\n')
print(text[0:612]) # show some text

TEXT LENGTH: 94687

TEXT SAMPLE:

i
from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou, contracted to thine own bright eyes,
feed'st thy light's flame with self-substantial fuel,
making a famine where abundance lies,
thy self thy foe, to thy sweet self too cruel:
thou that art now the world's fresh ornament,
and only herald to the gaudy spring,
within thine own bud buriest thy content,
and tender churl mak'st waste in niggarding:
pity the world, or else this glutton be,
to eat the world's due, by the grave and thee.


In [3]:
# created some dictionaries
chars = sorted(list(set(text)))
len_chars = len(chars)
print('total chars: ' + str(len_chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 38


The text is chunked up into sequences of uniform length. Each sequence is a feature and has a target corresponding to the character in the text immediately following the sequence. The features and targets are then vectorized. Each character is converted to a Boolean vectors having only one true element at the possiton corresponding to the character. <em>Note: making these vectorized features and targets integers (0s and 1s) instead of booleans seriously hampers learning and I am not sure why.</em>

In [4]:
# cut the text in semi-redundant sequences of maxlen characters
seq_length = 128
step = 3
features = []
targets = []
for i in range(0, len_text - seq_length, step):
    features.append(text[i: i + seq_length])
    targets.append(text[i + seq_length])
num_features = len(features)
print('number of features (sequences):'.upper() + str(num_features))

NUMBER OF FEATURES (SEQUENCES):31520


In [5]:
x = np.zeros((num_features, seq_length, len_chars), dtype=np.bool)
y = np.zeros((num_features, len_chars), dtype=np.bool)
for i, feature in enumerate(features):
    for t, char in enumerate(feature):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[targets[i]]] = 1

## Define the Model Graphs

Two RNN graphs are implemented here. One is more deep and less wide and the other is less deep and more wide. The model parameters have been tuned to give decent results. A function is also defined to allow sampling of the train modeled in order to generate new character sequences. In additoin, a function is defined which is called at the end of each epoch to generate and display generated character sequences as the network learns

In [6]:
def deep_model(seq_length, n_chars):
    print('buildinng two-layer model with 128 memory units...'.upper()+'\n')
    
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(seq_length, n_chars)))
    model.add(LSTM(128))
    model.add(Dropout(0.1))
    model.add(Dense(len(chars), activation='softmax'))

    optimizer = RMSprop(lr=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

def wide_model(seq_length, n_chars):
    print('building one-layer model with 256 memory units...'.upper()+'\n')
    
    model = Sequential()
    model.add(LSTM(256, input_shape=(seq_length, n_chars)))
    model.add(Dropout(0.1))
    model.add(Dense(len(chars), activation='softmax'))

    optimizer = RMSprop(lr=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

def sample(probs, method=0):
    probs = np.asarray(probs).astype('float64')
    probs = probs/np.sum(probs)
    
    # helper function to sample an index from a probability array
    if method == 0: 
        # method 0: just return index of character with the highest probability
        index = np.argmax(probs)
    elif method == 1: 
        # method 1: draw a random number from 0 to 1, calculate the cumulative sum of the prediction vector.
        # return the index of the first element in the cumulative sum greater than the random number
        index = np.argwhere(np.cumsum(probs)>np.random.uniform())[0][0]
    elif method == 2:
        # method 2: draw an element from a multinomial distribution defined by the given probabilities
        index = np.argmax(np.random.multinomial(1, probs, 1))
    elif method == 3:
        # method 3: emphasis larger probabilities and diminish smaller probabilities by doing a log transform
        temperature = 0.5 # less than one increases differences between small and large probabilities
        probs = np.log(probs) / temperature # same as method 2 with temperature = 1
        probs = np.exp(probs) # undo log transform
        probs = probs / np.sum(probs)
        index = np.argmax(np.random.multinomial(1, probs, 1))
        
    return index

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    if (epoch)%10 == 0:

        print('\nGENERATING CHARACTER SEQUENCE AFTER EPOCH: {}\n'.format(epoch+1))

        start_index = random.randint(0, len_text - seq_length - 1)
       
        
        seed = text[start_index: start_index + seq_length]
        print('GENERATING WITH SEED:\n\n' + seed + '\n')
        
        
        print('MAX SAMPLING:\n')
        phrase = seed
        generated = ''
        generated += phrase
        sys.stdout.write(generated)

        for i in range(seq_length):
            x_pred = np.zeros((1, seq_length, len_chars))
            for t, char in enumerate(phrase):
                x_pred[0, t, char_indices[char]] = 1.

            probs = model.predict(x_pred, verbose=0)[0]
            next_index = sample(probs, method=0)
            next_char = indices_char[next_index]

            generated += next_char
            phrase = phrase[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print('\n')
        
        print('PROBABILISTIC SAMPLING:\n')
        phrase = seed
        generated = ''
        generated += phrase
        sys.stdout.write(generated)

        for i in range(seq_length):
            x_pred = np.zeros((1, seq_length, len_chars))
            for t, char in enumerate(phrase):
                x_pred[0, t, char_indices[char]] = 1.

            probs = model.predict(x_pred, verbose=0)[0]
            next_index = sample(probs, method=1)
            next_char = indices_char[next_index]

            generated += next_char
            phrase = phrase[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print('\n')

## Model Training

In [7]:
# deeper, less wide, model
model = deep_model(seq_length, len_chars)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=256,
          epochs=41,
          # epochs=21, # for frankenstein use fewer epochs because the text is much longer
          callbacks=[print_callback])

BUILDINNG TWO-LAYER MODEL WITH 128 MEMORY UNITS...

Epoch 1/41
31520/31520 [==============================] - 26s 822us/step - loss: 2.8473 - acc: 0.2083

GENERATING CHARACTER SEQUENCE AFTER EPOCH: 1

GENERATING WITH SEED:

et this sad interim like the ocean be
which parts the shore, where two contracted new
come daily to the banks, that when they se

MAX SAMPLING:

et this sad interim like the ocean be
which parts the shore, where two contracted new
come daily to the banks, that when they ser than sor than sor than sor than sor than sor than sor than sor than sor than sor than sor than sor than sor than sor than sor 

PROBABILISTIC SAMPLING:

et this sad interim like the ocean be
which parts the shore, where two contracted new
come daily to the banks, that when they seoe, thad, o it woe tout on, oxd ghauu laree
fet favee lou si biags anua thoas,
coet ny wert poal,
huy lose sor youl tin ne fana 

Epoch 2/41
31520/31520 [==============================] - 25s 785us/step - loss: 2.2850 - 

In [8]:
# less deep but wider model
# deeper, less wide, model
model = wide_model(seq_length, len_chars)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=256,
          epochs=41,
          # epochs=21, # for frankenstein use fewer epochs because the text is much longer
          callbacks=[print_callback])

BUILDING ONE-LAYER MODEL WITH 256 MEMORY UNITS...

Epoch 1/41
31520/31520 [==============================] - 13s 409us/step - loss: 2.7907 - acc: 0.2268

GENERATING CHARACTER SEQUENCE AFTER EPOCH: 1

GENERATING WITH SEED:

redeem,
in gentle numbers time so idly spent;
sing to the ear that doth thy lays esteem
and gives thy pen both skill and argumen

MAX SAMPLING:

redeem,
in gentle numbers time so idly spent;
sing to the ear that doth thy lays esteem
and gives thy pen both skill and argumen the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

PROBABILISTIC SAMPLING:

redeem,
in gentle numbers time so idly spent;
sing to the ear that doth thy lays esteem
and gives thy pen both skill and argumenl'e heouf ans, meve,
tee leup beve; ar teve,
kn plam -ikeri.nges jethtres,
live ly un, whes,,
eore.
tolljeleee, areid ?fend ouse

Epoch 2/41
31520/31520 [==============================] - 12s 392us/step - loss: 2.2425 - a